In [1]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

from glob import glob
import math
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time
import json
import random
import pandas as pd
from tqdm import tqdm
import trimesh

import torch

from ANALYSIS.analysis_utils import (
    plot_panel_info,
    visualize_meshes_plotly,
    filter_segmentation_map,
    filter_segmentation_map_clusters,
    is_clockwise,
)

from env_constants import SEWFORMER_PROJ_ROOT, DATASET_ROOT, PYGARMENT_ROOT

sys.path.append(PYGARMENT_ROOT)

import pygarment as pyg

In [2]:
def filter_fn(path):
    if not os.path.isdir(path):
        print(f"{path} is not a directory")
        return False
    if not os.path.exists(os.path.join(path, "static", "spec_config.json")):
        print(f"No spec_config.json in {path}")
        return False
    return True

raw_combination_path_list = sorted(glob(os.path.join(DATASET_ROOT, "sewfactory", "*")))
combination_path_list = []
for garment_path in tqdm(raw_combination_path_list):
    if filter_fn(garment_path):
        combination_path_list.append(garment_path)
len(combination_path_list)

100%|██████████| 1954/1954 [00:00<00:00, 63318.32it/s]

No spec_config.json in /Users/hjp/HJP/KUAICV/VTO/DATASET/sewfactory/sewfactory/dress_sleeveless_M827MSUPRJ
No spec_config.json in /Users/hjp/HJP/KUAICV/VTO/DATASET/sewfactory/sewfactory/dress_sleeveless_ME7Y3S7MLB
No spec_config.json in /Users/hjp/HJP/KUAICV/VTO/DATASET/sewfactory/sewfactory/dress_sleeveless_MG7MML3QGT
No spec_config.json in /Users/hjp/HJP/KUAICV/VTO/DATASET/sewfactory/sewfactory/dress_sleeveless_NIJ4SIHR44
No spec_config.json in /Users/hjp/HJP/KUAICV/VTO/DATASET/sewfactory/sewfactory/dress_sleeveless_VIXKCSZRM5
No spec_config.json in /Users/hjp/HJP/KUAICV/VTO/DATASET/sewfactory/sewfactory/jumpsuit_sleeveless_GR6S0C83IQ
No spec_config.json in /Users/hjp/HJP/KUAICV/VTO/DATASET/sewfactory/sewfactory/jumpsuit_sleeveless_KBMBR543B1
No spec_config.json in /Users/hjp/HJP/KUAICV/VTO/DATASET/sewfactory/sewfactory/jumpsuit_sleeveless_KM5DJC1GHE
No spec_config.json in /Users/hjp/HJP/KUAICV/VTO/DATASET/sewfactory/sewfactory/jumpsuit_sleeveless_LCRSE6412S
No spec_config.json in /U

1879

In [3]:
import json
import numpy as np
import smplx
import torch
import trimesh

SMPLH_PATH = os.path.join(
    SEWFORMER_PROJ_ROOT, "Sewformer", "assets",
)

# Create SMPL-H model
model = smplx.create(
    model_path=SMPLH_PATH,
    model_type='smplh',  # Specifically use SMPL-H
    ext='pkl',
    gender='female',
    use_pca=False,  # Important: disable PCA for hand poses
    batch_size=1,
)

In [24]:
garment_name_dict = {} # check if some garment is duplicated
garment_name_list = []
matching_combination_name_list = []
panel_count_list = []
stitch_count_list = []
panel_name_contains_digit_list = []
contains_clockwise_panel_list = []
edge_type_list_list = []
mesh_len_segmentation_match_list = []
mesh_filter_failed_list = []
none_vertex_count_list = []
none_filtered_vertex_count_list = []
stitch_vertex_count_list = []
stitch_filtered_vertex_count_list = []

for combination_path in tqdm(combination_path_list) :
    spec_config_path = os.path.join(combination_path, "static", "spec_config.json")
    with open(spec_config_path, "r") as f:
        spec_config = json.load(open(spec_config_path, "r"))

    combination_garment_name_list = list(map(
        lambda x : os.path.basename(x["spec"].replace("\\", "/")),
        spec_config.values()
    ))

    try :
        # mesh_list = []
        for garment_name in combination_garment_name_list :
            if garment_name not in garment_name_dict :
                garment_name_dict[garment_name] = 0
            else :
                print(f"duplicating garment name {garment_name}")
                break
            garment_name_dict[garment_name] += 1
            


            spec_file_path = os.path.join(
                combination_path, "static", f"{garment_name}_specification.json"
            )
            pattern = pyg.pattern.wrappers.VisPattern(spec_file_path)
            drawn_pattern_list = list(map(
                lambda pannel_name : pattern._draw_a_panel(
                    pannel_name, apply_transform=False, fill=True
                ),
                pattern.panel_order()
            ))
            panel_svg_path_dict = {
                panel_name : pattern._draw_a_panel(
                    panel_name, apply_transform=False, fill=True
                )
                for panel_name in pattern.panel_order()
            }

            stitch_dict = {
                i : v for i, v in enumerate(pattern.pattern['stitches'])
            }

            # find if panel name contains digit
            panel_name_contains_digit = any(map(
                lambda x : any(char.isdigit() for char in x),
                pattern.panel_order()
            ))


            contains_clockwise_panel = False
            for panel_name, panel in panel_svg_path_dict.items() :
                point_list = []
                edges = list(panel[0])
                for edge in edges :
                    point_list.append((edge.start.real, edge.start.imag))
                if is_clockwise(point_list) :
                    contains_clockwise_panel = True
                    break
                
                
            edge_type_set = set()
            for panel_name, panel in panel_svg_path_dict.items() :
                for edge in panel :
                    if type(edge) not in edge_type_set :
                        edge_type_set.add(type(edge))

            mesh = trimesh.load_mesh(os.path.join(
                combination_path, "static", f"{garment_name}_{garment_name}.obj"
            ))

            with open(
                os.path.join(
                    combination_path, "static", f"{garment_name}_{garment_name}_segmentation.txt"
                ),
                "r"
            ) as f:
                mesh_segmentation_list = list(map(
                    lambda x : x.strip(),
                    f.readlines()
                ))
            
            
            if len(mesh_segmentation_list) != mesh.vertices.shape[0] :    
                print(f"{combination_path} {garment_name} {mesh.vertices.shape[0]} {len(mesh_segmentation_list)}")
                mesh_len_segmentation_match = False
            else :
                mesh_len_segmentation_match = True
                
            
            
            filtered_mesh_segmentation_list = filter_segmentation_map(          mesh, mesh_segmentation_list)
            filtered_mesh_segmentation_list = filter_segmentation_map_clusters( mesh, filtered_mesh_segmentation_list, threshold=80)
            filtered_mesh_segmentation_list = filter_segmentation_map(          mesh, filtered_mesh_segmentation_list)
            filtered_mesh_segmentation_list = filter_segmentation_map_clusters( mesh, filtered_mesh_segmentation_list, threshold=80)
            filtered_mesh_segmentation_list = filter_segmentation_map(          mesh, filtered_mesh_segmentation_list)
            filtered_mesh_segmentation_list = filter_segmentation_map_clusters( mesh, filtered_mesh_segmentation_list, threshold=80)
            filtered_mesh_segmentation_list = filter_segmentation_map(          mesh, filtered_mesh_segmentation_list)
            
            vertex_mask_dict = {}
            for panel_name in panel_svg_path_dict.keys() :
                vertex_mask_dict[panel_name] = np.array(list(map(
                    lambda x : x == panel_name,
                    mesh_segmentation_list
                )))
            vertex_mask_dict["stitch"] = np.array(list(map(
                lambda x : x == "stitch",
                mesh_segmentation_list
            )))
            vertex_mask_dict["None"] = np.array(list(map(
                lambda x : x == "None",
                mesh_segmentation_list
            )))
            

            filtered_vertex_mask_dict = {}
            for panel_name in panel_svg_path_dict.keys() :
                filtered_vertex_mask_dict[panel_name] = np.array(list(map(
                    lambda x : x == panel_name,
                    filtered_mesh_segmentation_list
                )))
            
            mesh_filter_failed = False
            
            stitch_mask = np.array(list(map(
                lambda x : x == "stitch",
                filtered_mesh_segmentation_list
            )))
            if stitch_mask.sum() > 0 :
                mesh_filter_failed = True
                filtered_vertex_mask_dict["stitch"] = stitch_mask
            
            none_mask = np.array(list(map(
                lambda x : x == "None",
                filtered_mesh_segmentation_list
            )))
            if none_mask.sum() > 0 :
                mesh_filter_failed = True
                filtered_vertex_mask_dict["None"] = none_mask
            
            
            with open(
                os.path.join(
                    combination_path, "static", f"{garment_name}_{garment_name}_segmentation_filtered.txt"
                ),
                "w"
            ) as f :
                f.write("\n".join(filtered_mesh_segmentation_list))
                f.write("\n")
        
            
            garment_name_list.append(garment_name)
            matching_combination_name_list.append(os.path.basename(combination_path))
            panel_count_list.append(len(panel_svg_path_dict))
            stitch_count_list.append(len(stitch_dict))
            panel_name_contains_digit_list.append(panel_name_contains_digit)
            contains_clockwise_panel_list.append(contains_clockwise_panel)
            edge_type_list_list.append(list(edge_type_set))
            mesh_len_segmentation_match_list.append(mesh_len_segmentation_match)
            none_vertex_count_list.append(
                (np.array(mesh_segmentation_list) == "None").sum()
            )
            stitch_vertex_count_list.append(
                (np.array(mesh_segmentation_list) == "stitch").sum()
            )
            stitch_filtered_vertex_count_list.append(
                stitch_mask.sum()
            )
            none_filtered_vertex_count_list.append(
                none_mask.sum()
            )
            mesh_filter_failed_list.append(mesh_filter_failed)
    except Exception as e :
        print(f"{combination_path}")
        print(f"{garment_name}")
        print(f"{e}")
        print()

    
garment_df = pd.DataFrame(
    {
        "garment_name" : garment_name_list,
        "matching_combination_name" : matching_combination_name_list,
        "panel_count" : panel_count_list,
        "stitch_count" : stitch_count_list,
        "panel_name_contains_digit" : panel_name_contains_digit_list,
        "contains_clockwise_panel" : contains_clockwise_panel_list,
        # "edge_type_list" : edge_type_list_list,
        "mesh_len_segmentation_match" : mesh_len_segmentation_match_list,   
        "mesh_filter_failed" : mesh_filter_failed_list,
        "none_vertex_count" : none_vertex_count_list,
        "stitch_vertex_count" : stitch_vertex_count_list,
        "none_filtered_vertex_count" : none_filtered_vertex_count_list,
        "stitch_filtered_vertex_count" : stitch_filtered_vertex_count_list,
    }
)
garment_df.to_csv(
    f"garment_df_{time.strftime('%Y%m%d_%H%M%S')}.csv",
    index=False
)
garment_df

 21%|██        | 398/1879 [06:01<13:07,  1.88it/s]

/Users/hjp/HJP/KUAICV/VTO/DATASET/sewfactory/sewfactory/jumpsuit_sleeveless_BSCJNM8VBX jumpsuit_sleeveless_BSCJNM8VBX 10842 10843
/Users/hjp/HJP/KUAICV/VTO/DATASET/sewfactory/sewfactory/jumpsuit_sleeveless_BSCJNM8VBX
jumpsuit_sleeveless_BSCJNM8VBX
list index out of range



 24%|██▍       | 458/1879 [06:49<13:29,  1.75it/s]

/Users/hjp/HJP/KUAICV/VTO/DATASET/sewfactory/sewfactory/jumpsuit_sleeveless_EVYB2P4PN2 jumpsuit_sleeveless_EVYB2P4PN2 10854 10855
/Users/hjp/HJP/KUAICV/VTO/DATASET/sewfactory/sewfactory/jumpsuit_sleeveless_EVYB2P4PN2
jumpsuit_sleeveless_EVYB2P4PN2
list index out of range



 30%|███       | 566/1879 [08:07<12:23,  1.77it/s]

...

/Users/hjp/HJP/KUAICV/VTO/DATASET/sewfactory/sewfactory/wb_dress_sleeveless_AN2FZQJEC7
wb_dress_sleeveless_AN2FZQJEC7
[Errno 2] No such file or directory: '/Users/hjp/HJP/KUAICV/VTO/DATASET/sewfactory/sewfactory/wb_dress_sleeveless_AN2FZQJEC7/static/wb_dress_sleeveless_AN2FZQJEC7_specification.json'



 92%|█████████▏| 1728/1879 [27:40<03:04,  1.22s/it]

/Users/hjp/HJP/KUAICV/VTO/DATASET/sewfactory/sewfactory/wb_dress_sleeveless_FE6NJ0TK3A
wb_dress_sleeveless_FE6NJ0TK3A
[Errno 2] No such file or directory: '/Users/hjp/HJP/KUAICV/VTO/DATASET/sewfactory/sewfactory/wb_dress_sleeveless_FE6NJ0TK3A/static/wb_dress_sleeveless_FE6NJ0TK3A_specification.json'



100%|██████████| 1879/1879 [30:24<00:00,  1.03it/s]


,garment_name,matching_combination_name,panel_count,stitch_count,panel_name_contains_digit,contains_clockwise_panel,mesh_len_segmentation_match,mesh_filter_failed,none_vertex_count,stitch_vertex_count,none_filtered_vertex_count,stitch_filtered_vertex_count
0,dress_sleeveless_02OIQPCOWU,dress_sleeveless_02OIQPCOWU,4,10,False,False,True,False,238,234,0,0
1,dress_sleeveless_09OPJBWSO8,dress_sleeveless_09OPJBWSO8,4,10,False,False,True,False,244,240,0,0
2,dress_sleeveless_09TBE1ATRH,dress_sleeveless_09TBE1ATRH,4,10,False,False,True,False,232,228,0,0
3,dress_sleeveless_0C4VMNOCLU,dress_sleeveless_0C4VMNOCLU,4,10,False,False,True,False,274,270,0,0
4,dress_sleeveless_0DAY6M9ZYC,dress_sleeveless_0DAY6M9ZYC,4,10,False,False,True,False,242,238,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2520,wb_dress_sleeveless_Z1IW81L0N1,wb_dress_sleeveless_Z1IW81L0N1,6,12,False,False,True,False,380,372,0,0
2521,wb_dress_sleeveless_Z9G8LZ1IN0,wb_dress_sleeveless_Z9G8LZ1IN0,6,12,False,False,True,False,416,408,0,0
2522,wb_dress_sleeveless_ZQE9US31KP,wb_dress_sleeveless_ZQE9US31KP,6,12,False,False,True,False,386,378,0,0
2523,wb_dress_sleeveless_ZVM7I1QKYL,wb_dress_sleeveless_ZVM7I1QKYL,6,12,False,False,True,False,344,336,0,0


In [23]:
print(garment_df["mesh_filter_failed"].sum())
print((garment_df["none_filtered_vertex_count"] > 0).sum())
print((garment_df["stitch_filtered_vertex_count"] > 0).sum())

# 268
# 23
# 251

268
23
251


In [ ]:
def detect_stitch_vertices(mesh, vertex_labels):
    """
    Reclassifies boundary vertices as stitches if they are between two or more distinct panels.

    Parameters:
    mesh: trimesh.Trimesh - The garment mesh
    vertex_labels: np.array - Array of panel labels for each vertex

    Returns:
    updated_labels: np.array - Updated labels with stitch vertices reassigned
    """

    updated_labels = vertex_labels.copy()  # Copy original labels

    for v_idx in range(len(mesh.vertices)):  # Iterate through all vertices
        neighbors = mesh.vertex_neighbors[v_idx]  # Get neighboring vertices
        neighbor_labels = set(vertex_labels[n] for n in neighbors)  # Collect unique panel labels
        
        if len(neighbor_labels) > 1:  # More than one unique panel => stitch
            updated_labels[v_idx] = "stitch"

    return updated_labels

In [21]:
filtered_combination_path_list = np.unique(garment_df[
    garment_df["mesh_filter_failed"] == False
]["matching_combination_name"]).tolist()

1633